# 一篇搞定 OPENAI 的 Function calling

> 函数 （Function Calling) 调用提供了一种将 GPT 的能力与外部工具和 API 相连接的新方法。

在这个文章中，我想向您展示 OpenAI 模型的函数 （Function Calling) 调用能力，并向您展示如何将此新功能与 Langchain 集成。

我将通过以下代码详细介绍这个工作原理，开始吧！

In [ ]:
%pip install --upgrade langchain
%pip install python-dotenv
%pip install openai

### 安装包

In [3]:
import pkg_resources


def print_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f" The version of the {package_name} library is {version}.")
    except pkg_resources.DistributionNotFound:
        print(f" The {package_name} library is not installed.")


print_version(" langchain")

The version of the langchain library is 0.0.221.


### 连接 OpenAI

In [4]:
from dotenv import load_dotenv
import os
import openai
import json

load_dotenv('.env.local')
openai.api_key = os.environ.get(" OPENAI_API_KEY")

### 定义一个 Function Calling

In [5]:
def get_pizza_info(pizza_name: str):
    pizza_info = {
        " name": pizza_name,
        " price": " 10.99",
    }
    return json.dumps(pizza_info)

In [6]:
 functions = [
 
    {
        " name": " get_pizza_info",
        " description": " Get name and price of a pizza of the restaurant",
        " parameters": {
            " type": " object",
            " properties": {
                " pizza_name": {
                    " type": " string",
                    " description": " The name of the pizza, e.g. Salami",
                },
            },
            " required": ["pizza_name"],
        },
    }
]

### 运行第一个 Function Calling

In [7]:
def chat(query):
    response = openai.ChatCompletion.create(
        model =" gpt-3.5-turbo-0613",
        messages = [{"role": "user", "content": query}],
        functions = functions,
    )
    message = response ["choices"][0] ["message"]
    return message

In [8]:
chat(" What is the capital of france?")

<OpenAIObject at 0x166d47a42f0> JSON: {
  " role": " assistant",
  " content": " The capital of France is Paris."
}

In [9]:
query = " How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x166d47a43b0> JSON: {
  " role": " assistant",
  " content": null,
  " function_call": {
    " name": " get_pizza_info",
    " arguments": "{\n\"pizza_name\": \"Salami\"\n}"
  }
}

In [10]:
if message.get(" function_call"):
    # 解析第一次调用的时候返回的 pizza 信息
    function_name = message ["function_call"]["name"]
    pizza_name = json.loads(message ["function_call"] ["arguments"]).get(" pizza_name")
    print(pizza_name)
    # 这里将 chat 小助手函数的响应结果提取后，传递 function_response
    function_response = get_pizza_info(
        pizza_name = pizza_name 
    )

    second_response = openai.ChatCompletion.create(
        model =" gpt-3.5-turbo-0613",
        messages =[
 
            {" role": " user", " content": query},
            message,
            {
                " role": " function",
                " name": function_name,
                " content": function_response, # function calling 的 content 是 get_pizza_info 函数 
            },
        ],
    )

second_response

Salami


<OpenAIObject chat.completion id=chatcmpl-7Y9045lCV15L1psS5SNYclk4SGcDU at 0x166c574fa10> JSON: {
  " id": " chatcmpl-7Y9045lCV15L1psS5SNYclk4SGcDU",
  " object": " chat.completion",
  " created": 1688372104,
  " model": " gpt-3.5-turbo-0613",
  " choices": [
 
    {
      " index": 0,
      " message": {
        " role": " assistant",
        " content": " The cost of a pizza salami is $10.99."
 
      },
      " finish_reason": " stop"
    }
  ],
  " usage": {
    " prompt_tokens": 58,
    " completion_tokens": 13,
    " total_tokens": 71
  }
}

## 如何与 LangChain 一起使用?

首先导入 ChatOpenAI 类和 HumanMessage、AIMessage，还有 ChatMessage 类，这些类可以帮助我们创建这种功能，包括用户角色等。
我们可以不必要像之前那样，定义角色等，只需要传递 `content`。其他的都交给了 Langchain.

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

先问问法国的首都，测试：What is the capital of france?

In [13]:
llm = ChatOpenAI(model =" gpt-3.5-turbo-0613")
message = llm.predict_messages(
    [HumanMessage(content ="What is the capital of france?")], functions = functions
)
message

AIMessage(content ='The capital of France is Paris.', additional_kwargs ={}, example = False)

现在我们再次运行查询披萨莎拉米在餐厅里的价格。How much does pizza salami cost?

In [19]:
llm = ChatOpenAI(model =" gpt-3.5-turbo-0613")
message_pizza = llm.predict_messages(
    [HumanMessage(content ="How much does pizza salami cost?")], functions = functions
)
message

AIMessage(content ='', additional_kwargs ={'function_call': {'name': 'get_pizza_info', 'arguments': '{\n" pizza_name": " Salami"\n}'}}, example = False)

In [21]:
import json

pizza_name = json.loads(message_pizza.additional_kwargs ["function_call"]["arguments"]).get(" pizza_name")
pizza_name

'Salami'

然后我们再次调用这个函数 （Function Calling) ，就像这样再次调用函数 （Function Calling) ，我们得到名称莎拉米和价格为 10.99。

In [23]:
pizza_api_response = get_pizza_info(pizza_name = pizza_name)
pizza_api_response

'{" name": " Salami", " price": " 10.99"}'

我们再次运行这个，我们应该得到与之前类似的响应，`AIMessage` 内容是在餐厅里的披萨价格为 `10.99`，因此非常容易使用.

In [24]:
second_response = llm.predict_messages(
    [
 
        HumanMessage(content = query),
        AIMessage(content = str(message_pizza.additional_kwargs)),
        ChatMessage(
            role =" function",
            additional_kwargs ={
                " name": message_pizza.additional_kwargs ["function_call"]["name"]
            },
            content = pizza_api_response
        ),
    ],
    functions = functions,
)
second_response

 AIMessage(content ='The pizza Salami costs $10.99.', additional_kwargs ={}, example = False)

### 使用 LangChain 的 tools

LangChain 已经提供了与外部世界交互的另一种标准化方法，以进行请求或其他操作，这些称为工具 tools，工具 tools 是由 Chain 提供的类，您也可以创建自己的工具，我将向您展示如何做到这一点。

In [25]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class StupidJokeTool(BaseTool):
    name = " StupidJokeTool"
    description = " Tool to explain jokes about chickens"

    def _run(
 
        self, query: str, run_manager: Optional [CallbackManagerForToolRun] = None
    ) -> str:
        return " It is funny, because AI..."

    async def _arun(
 
        self, query: str, run_manager: Optional [AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """ Use the tool asynchronously."""
        raise NotImplementedError(" joke tool does not support async")

如果您有了自己的工具与类一起使用，您可以轻松将自己的类转换为格式化的工具: `format_tool_to_openai_function`，我还在这里导入了 `MoveFileTool` 工具，它允许您在计算机上移动文件。

In [27]:
from langchain.tools import format_tool_to_openai_function, MoveFileTool


tools = [StupidJokeTool(), MoveFileTool()]
# 将自己的 tools 转换为格式化的 function
functions = [format_tool_to_openai_function(t) for t in tools]
# functions 是之前定义的一个变量：一个函数列表

query = " Why does the chicken cross the road? To get to the other side"
output = llm.predict_messages([HumanMessage(content = query)], functions = functions)
output

AIMessage(content ='', additional_kwargs ={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n"__arg1": " To get to the other side"\n}'}}, example = False)


现在我们可以再次像这样使用它，例如为什么鸡过马路？

In [32]:
query = " Why does the chicken cross the road? To get to the other side"
second_response = llm.predict_messages(
    [
 
        HumanMessage(content = query),
        AIMessage(content = str(output.additional_kwargs)),
        ChatMessage(
            role =" function",
            additional_kwargs ={
                " name": output.additional_kwargs ["function_call"]["name"]
            },
            content ="""
 
                {tool_response}
            """,
 
        ),
    ],
    functions = functions,
)
second_response

AIMessage(content ='', additional_kwargs ={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n  "__arg1": " To get to the other side"\n}'}}, example = False)

### Langchain Agent 如何实现 Function Calling ？

我将向您展示它是如何工作的，首先我们导入一些链 Chain，例如 `LLMMathChain`，还有一个 `chat_models`，聊天模型在这里使用 `ChatOpenAI` 创建我们的 LLM。

In [33]:
from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature = 0, model =" gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm = llm, verbose = True)
 tools = [
 
    Tool(
        name =" Calculator",
        func = llm_math_chain.run,
        description =" useful for when you need to answer questions about math"
    ),
]

In [34]:
agent = initialize_agent(tools, llm, agent = AgentType.OPENAI_FUNCTIONS, verbose = True)

现在我们运行“法国的首都是什么”，我们得到的结果是法国的首都：巴黎。

In [35]:
agent.run(" What is the capital of france?")



 > Entering new  chain...
 
[32; 1m[1; 3mThe capital of France is Paris.
 

 > Finished chain.
 

'The capital of France is Paris.'

如果我们想知道 100 除以 25 等于多少，这时候计算器被调用，我们得到最终答案 100 除以 25 等于 4。

In [36]:
agent.run("100 除以 25 等于多少?")



 > Entering new  chain...
 
[32; 1m[1; 3m
 
Invoking: `Calculator` with `100 / 25`



 

 > Entering new  chain...
 
 100 / 25[32; 1m[1; 3m```text
 
100 / 25
 ```
...numexpr.evaluate("100 / 25")...

Answer: 4.0
> Finished chain.
Answer: 4.0100 除以 25 等于 4。

> Finished chain.


'100 除以 25 等于 4。'